In [11]:
from music21 import *
from comp2_markov import *

In [12]:
melody_in: stream.Stream = parse_midi("example_inputs/finalcountdown.mid")
melody_in = melody_in.transpose(-1)

output_path = '/mnt/c/Users/jwest/Desktop/algocomps/comp2-markov/generations/'

print("Original melody:")
melody_in.show('midi');
melody_in.write('midi', output_path + 'original.mid')

Original melody:


'/mnt/c/Users/jwest/Desktop/algocomps/comp2-markov/generations/original.mid'

In [13]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['v', 'i5#2', 'iv5#2', 'iv5#2', 'v+64', 'i5#2', 'iv5#2', 'iv5#2', 'v+64', 'i5#2', 'iv5#2', 'iv5#2', 'v+64', 'iv5#2', 'v', 'v']


In [14]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')

In [15]:
markov_table = generate_markov_table()
substituted_chords = substitute_chords(simple_triads, markov_table)
for pos_i in range(5):
    substituted_chords = substitute_chords(substituted_chords, markov_table)

out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, substituted_chords)

out_score.show('midi')

Total number of states: 2744

new chords:
['v7', 'i', 'ivsus4', 'II76', 'VI', 'i2', 'v', 'ii', 'v7', 'I2', 'vi2', 'vii7', 'v2', 'iv76', 'v2', 'v2']

new chords:
['v7', 'i2', 'II2', 'VIIsus4', 'i', 'v2', 'v2', 'ii', 'v', 'i2', 'iv2', 'v2', 'v2', 'ii2', 'v7', 'IV2']

new chords:
['VII2', 'v2', 'II2', 'vii7', 'i76', 'v7', 'v', 'ii2', 'v', 'i', 'ii7', 'v2', 'v', 'ii2', 'v', 'i2']

new chords:
['I2', 'v76', 'II6', 'V7', 'v', 'ii', 'v2', 'iv2', 'v2', 'I', 'iv7', 'v2', 'i2', 'ii', 'v6', 'i2']

new chords:
['I7', 'v7', 'II2', 'V2', 'I7', 'ii76', 'v2', 'i', 'v', 'IV2', 'IV2', 'v', 'isus4', 'ii7', 'v2', 'i2']

new chords:
['ii2', 'v7', 'i6', 'V2', 'I2', 'v2', 'III2', 'i2', 'VII76', 'ii2', 'II', 'v7', 'i76', 'ii2', 'v76', 'i7']


In [ ]:
import torch
from preprocess import *
from bigram import BigramModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_save_path = '/mnt/c/Users/jwest/Desktop/algocomps/comp2-markov/trained_models/parkergpt'
checkpoint_f = torch.load(model_save_path + '_forward.pt', map_location=device)
checkpoint_b = torch.load(model_save_path + '_backward.pt', map_location=device)

model_f = BigramModel(
    device=device,
    block_size=checkpoint_f['block_size'],
    vocab_size=checkpoint_f['vocab_size'],
    n_embed=checkpoint_f['n_embed']
)

model_b = BigramModel(
    device=device,
    block_size=checkpoint_b['block_size'],
    vocab_size=checkpoint_b['vocab_size'],
    n_embed=checkpoint_b['n_embed']
)

# Load the saved weights
model_f.load_state_dict(checkpoint_b['model_state_dict'])
model_b.load_state_dict(checkpoint_b['model_state_dict'])

<All keys matched successfully>

In [17]:
gen_tokens_fwd = model_f.generate(
    idx=torch.zeros((1, 1), dtype=torch.long).to(device),
    max_length=100  # Generate sequence
)[0].tolist()

# Convert tokens to MIDI
gen_stream = sequence_to_score(tokens_to_sequence(gen_tokens_fwd))
gen_stream.show('midi')

In [18]:
def splice_melody(melody: stream.Stream,
                  past_context=6, # in number of notes or rests
                  future_context=8, # in timesteps: 24ths of a quarter note
                  max_tokens=20,
                  segment_length=6,
                  crossover_threshold=24) -> stream.Stream:
    
    len_split = segment_length * 24
    out = []
    out_fwd = []
    out_back = []
    pos = 0  # position in melody (in 24ths of a quarter note)
    pos_i = 0 # left boundary for current segment
    pos_j = len_split # right boundary
    idx = 0  # idx for forward model
    take = 'melody'
    
    while pos < melody.duration.quarterLength * 24:
        if take == 'melody':
            if pos_i > len_split:
                # print("end melody segment: len_split")
                # use the melody segments surrounding the slice to generate context
                # generate separate contexts for each models
                context_forward = make_tokens(score_to_sequence(out[-past_context:]))

                # use element index to get future context for backward model
                context_backward = make_tokens(score_to_sequence(
                    melody.flatten().notesAndRests.getElementsByOffset(
                        ((pos + len_split) / 24) + 1,
                        ((pos + len_split) / 24) + future_context
                    )[::-1] # reverse the context
                ))
                # if backward context is empty, pad with rests
                if len(context_backward) == 0:
                    final_rests = []
                    for i in range(future_context):
                        final_rests.append(note.Rest(quarterLength=1))
                    context_backward = make_tokens(score_to_sequence(final_rests))

                gen_tokens_fwd = sequence_to_score(tokens_to_sequence(model_f.generate(
                    idx=torch.tensor(context_forward, dtype=torch.long).unsqueeze(0).to(device),
                    max_length=max_tokens
                )[0].tolist())).flatten().notesAndRests

                gen_tokens_back = sequence_to_score(tokens_to_sequence(model_b.generate(
                    idx=torch.tensor(context_backward, dtype=torch.long).unsqueeze(0).to(device),
                    max_length=max_tokens
                )[0].tolist())[::-1]).flatten().notesAndRests

                # print("generated tokens")
                pos_i = pos
                pos_j = pos + len_split
                out_fwd = []
                out_back = []
                idx = 0
                take = 'model'
                continue

            # take next element from melody
            elem_fwd = melody.flatten().getElementAtOrBefore(pos / 24)
            if elem_fwd is None:
                break
            out.append(elem_fwd)
            pos += int(elem_fwd.quarterLength * 24)
            pos_i += int(elem_fwd.quarterLength * 24)
            
        else:
            if pos_j - pos_i < crossover_threshold:
                # print("end generated segment: crossover.")
                # print(f"out_fwd: [{len(out_fwd)}]")
                # print("middle rest duration:", max(0, pos_j - pos_i) / 24)
                # print(f"out_back: [{len(out_back)}]")
                middlerest_dur = max(0, pos_j - pos_i) / 24
                for i in range(len(out_back)):
                    out_back[i].offset -= middlerest_dur

                out.extend(out_fwd)
                out.append(note.Rest(quarterLength=middlerest_dur))
                out.extend(out_back)
                # print(f"out: [{len(out)}]")
                pos_i = 0
                pos += len_split
                take = 'melody'
                continue
                
            # Get next elem by index
            elem_fwd = gen_tokens_fwd[idx]
            
            # Create a copy of the element to prevent ID conflicts
            if isinstance(elem_fwd, note.Note):
                elem_fwd = note.Note(pitch=elem_fwd.pitch, quarterLength=elem_fwd.quarterLength)
            elif isinstance(elem_fwd, note.Rest):
                elem_fwd = note.Rest(quarterLength=elem_fwd.quarterLength)
            
            elem_fwd.offset = pos_i / 24 # Set offset
            pos_i += int(elem_fwd.quarterLength * 24) # Then increment position
            out_fwd.append(elem_fwd) # Then append

            elem_back = gen_tokens_back[idx]
            if isinstance(elem_back, note.Note):
                elem_back = note.Note(pitch=elem_back.pitch, quarterLength=elem_back.quarterLength)
            elif isinstance(elem_back, note.Rest):
                elem_back = note.Rest(quarterLength=elem_back.quarterLength)
            
            pos_j -= int(elem_back.quarterLength * 24) # Decrement position
            elem_back.offset = pos_j / 24 # Then set offset
            out_back.insert(0, elem_back) # Then append

            idx += 1  # Increment index for both streams
            if idx >= max_tokens:
                # print("end generated segment: token limit reached.")
                # print(f"out_fwd: {out_fwd}")
                # print(f"out_back: {out_back}")
                out.extend(out_fwd + out_back)
                # print(f"out: {out}")
                pos += len_split
                take = 'melody'
                continue
    
    return stream.Stream(out)


In [19]:
res_melody = splice_melody(melody_in,
                           past_context=3,
                           future_context=2,
                           max_tokens=20,
                           segment_length = 9,
                           crossover_threshold=25)

out_score = stream.Score()
out_score.insert(0, res_melody)
# out_score.insert(0, simple_triads.transpose(-12))

out_score.show('midi')

In [20]:
out_score.write('midi', output_path + 'output_spliced.mid')

'/mnt/c/Users/jwest/Desktop/algocomps/comp2-markov/generations/output_spliced.mid'